Deffuant model. Save only nb of groups, groups size and convergence time.

In [1]:
import numpy as np
from tqdm.notebook import tqdm
import csv
import random
from itertools import groupby
from random import shuffle

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')
from utils import *
from topological_metrics import get_weighted_graph

In [2]:
dataset = "pr_school";gap = 300;state_gap = 1200;nb_states_L = 3;nb_states_e = 8
#dataset = "h_school11";gap = 1200;state_gap = 1200;nb_states_L = 3;nb_states_e = 7
#dataset = "h_school13";gap = 1200;state_gap = 1200;nb_states_L = 4;nb_states_e = 29
#dataset = "InVS15"; gap = 3600; state_gap = 3600; nb_states_L = 3;nb_states_e = 12
#dataset = "hypertext";gap = 1200;state_gap = 1200;nb_states_L = 3;nb_states_e = 20

In [3]:
dataset

'pr_school'

# Functions

In [4]:
def upload_edgelist_D(dataset,gap):

    edgelist = pd.read_csv("../../Datasets/"+dataset+".dat",sep=" ",names=["t","i","j"])
    nb_nodes = max(max(edgelist.i),max(edgelist.j)) + 1
    nodes_list = np.arange(0,nb_nodes)
    times = [int(x/gap) for x in edgelist.t]
    edgelist.t = times

    return nodes_list,edgelist

In [5]:
def upload_edgelist_surrogate(dataset,net,net_idx,nb_states_e,gap,nb_states_L):
    path = '../../Generated_graphs/d2/'
    if net == 'E':
        directory = path+"%s_E_gap_%d_%d_states_e/"%(dataset,gap,nb_states_e)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])

    elif net == 'ES':       
        directory = path+"%s_ES_gap_%d_%d_states_e_%d_states_L/"%(dataset,gap,nb_states_e,nb_states_L)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])

    elif net == 'EST':       
        directory = path+"%s_EST_gap_%d_%d_states_e_%d_states_L/"%(dataset,gap,nb_states_e,nb_states_L)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])
    
    nb_nodes = max(max(edgelist.i),max(edgelist.j)) + 1
    nodes_list = np.arange(0,nb_nodes)
    times = [int(x/gap) for x in edgelist.t]
    edgelist.t = times

    return nodes_list,edgelist

In [6]:
def obtain_edgelist_dict(data_):
    edgelist_dict = {}
    for t in range(max(data_.t) + 1):
        edgelist_dict[t] = []
    
    for _, row in data_.iterrows():
        if row['t'] in edgelist_dict:
            if [row['i'],row['j']] not in edgelist_dict[row['t']]:
                edgelist_dict[row['t']].append([row['i'],row['j']])
            if [row['j'],row['i']] in edgelist_dict[row['t']] and row['i'] != row['j']:
                print('error1')
        else:
            print('error2')
    return edgelist_dict

In [7]:
def simulate(q, t0, edgelist_t,Gw,time_ev_name):
    b_max = max(edgelist_t.keys())
    b = 0
    t_interval_save = 5
    #Initialize time 0
    x = list(np.random.uniform(low=0, high=1, size=len(nodes_list)))
    t = t0
    t_conv = 0
    for times in range(100000):
        old_x = x.copy()
        shuffle(edgelist_t[t])
        for edge in edgelist_t[t]:
            
            if np.abs(x[edge[0]] - x[edge[1]]) < q:
                mid_x = (x[edge[0]] + x[edge[1]])/2
                x[edge[0]] = mid_x
                x[edge[1]] = mid_x
                
        if time_ev_name != None:
                save_on_csv(time_ev_name,x,'a')
                
        if not t_conv%t_interval_save:
            
            flag = False
            for edge in Gw.edges():
                xdiff = np.abs(x[edge[0]] - x[edge[1]])
                if xdiff < q and xdiff > 0.001:
                    flag = True
            if flag == False:
                break
            
        t += 1
        if t > max(edgelist_t.keys()):
            t = 0  
        t_conv += 1

    return x, t_conv

In [8]:
def all_equal(iterable):
    g = groupby(iterable)
    return next(g, True) and not next(g, False)

# Simulate

Choose if save also time evolution or not

In [9]:
save_time_ev = True

In [10]:
if save_time_ev:
    q_range = [0.1, 0.2, 0.3, 0.4]
else:
    q_range = np.linspace(0.1,0.55,num=10)

In [11]:
folder = 'results/' # to save results
if not os.path.exists(folder):
    os.makedirs(folder)

In [12]:
nb_surr = 10 # nb of generated networks
if save_time_ev:
    nb_runs_surr = 1
else:
    nb_runs_surr = 20 # nb of simulations on each network (so 200 simulations in total)
nb_runs_D = nb_surr*nb_runs_surr

D

In [13]:
net = 'D'

# Load edgelist
nodes_list,edgelist = upload_edgelist_D(dataset,gap)
Gw = get_weighted_graph(build_graphs(edgelist,gap))
edgelist_dict = obtain_edgelist_dict(edgelist)
non_empty_t = np.unique(edgelist.t)



# Simulate
for q in q_range:
    print('q',q)
    # save also time evolution or not:
    if save_time_ev:
        time_ev_name = "results/time_evol_t_save_1/x_t_%s_%s_q_%.2f.csv"%(dataset,net,q)
    else:
        time_ev_name = None
    
    filename_x = "results/xf_%s_%s_q_%.2f.csv"%(dataset,net,q) # opinions
    filename_t = "results/t_conv_%s_%s_q_%.2f.txt"%(dataset,net,q) # converegence time
    file_t = open(filename_t, 'a+')
    for run in range(nb_runs_D):
        t0 = random.choice(non_empty_t)
        x, t_conv = simulate(q, t0, edgelist_dict,Gw,time_ev_name)
        save_on_csv(filename_x,x,'a')
        file_t.write('%d\n'%t_conv)
    file_t.close()

q 0.1
q 0.2
q 0.3
q 0.4


Surrogates

In [14]:
for net in ['E','ES','EST']:
    print(net)
    for net_idx in tqdm(range(nb_surr)):
        print('net_idx',net_idx)
        # Load edgelist   
        nodes_list,edgelist = upload_edgelist_surrogate(dataset,net,net_idx,nb_states_e,gap,nb_states_L)
        Gw = get_weighted_graph(build_graphs(edgelist,gap))
        edgelist_dict = obtain_edgelist_dict(edgelist)

        non_empty_t = np.unique(edgelist.t)
        
        # Simulate
        for q in q_range:
            # save also time evolution or not:
            if save_time_ev:
                time_ev_name = "results/time_evol_t_save_1/x_t_%s_%s_q_%.2f.csv"%(dataset,net,q)
            else:
                time_ev_name = None
                
            filename_x = "results/xf_%s_%s_q_%.2f_%d.csv"%(dataset,net,q,net_idx) # opinions
            filename_t = "results/t_conv_%s_%s_q_%.2f.txt"%(dataset,net,q) # converegence time
            file_t = open(filename_t, 'a+')
            for run in range(nb_runs_surr):
                t0 = random.choice(non_empty_t)
                x, t_conv = simulate(q, t0, edgelist_dict,Gw,time_ev_name)
                save_on_csv(filename_x,x,'a')
                file_t.write('%d\n'%t_conv)
            file_t.close()

E


  0%|          | 0/10 [00:00<?, ?it/s]

net_idx 0
net_idx 1
net_idx 2
net_idx 3
net_idx 4
net_idx 5
net_idx 6
net_idx 7
net_idx 8
net_idx 9
ES


  0%|          | 0/10 [00:00<?, ?it/s]

net_idx 0
net_idx 1
net_idx 2
net_idx 3
net_idx 4
net_idx 5
net_idx 6
net_idx 7
net_idx 8
net_idx 9
EST


  0%|          | 0/10 [00:00<?, ?it/s]

net_idx 0
net_idx 1
net_idx 2
net_idx 3
net_idx 4
net_idx 5
net_idx 6
net_idx 7
net_idx 8
net_idx 9
